In [0]:
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, Activation, Flatten
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization

In [0]:
import sklearn
from sklearn.utils import shuffle
from datetime import datetime
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy import stats
import matplotlib.pyplot as plt

In [0]:
#import the csv
data = pd.read_csv('bpi_12_w_no_repeat.csv')
np.sum(data.isna())
# data['CompleteTimestamp'][0].day
d = data

In [508]:
data['CaseID'] = data['CaseID'] - 173688 
data.head(10)

,CaseID,ActivityID,CompleteTimestamp
0,0,3,2011-10-01 19:45:13
1,0,5,2011-10-01 20:17:08
2,0,6,2011-10-13 18:37:37
3,3,3,2011-10-01 19:43:13
4,3,5,2011-10-01 22:36:25
5,3,6,2011-10-10 19:30:54
6,3,6,2011-10-10 22:17:34
7,6,3,2011-10-01 19:35:59
8,6,5,2011-10-03 21:44:21
9,6,6,2011-11-05 1:05:01


In [509]:
#given the caseID, array 'a' outputs the number of events in  that particular caseID trace.
caseS = pd.Series(data['CaseID'] )
a = caseS.value_counts()
print("Number of events in trace caseID 0 :" + str(a[0]))
a[3]

Number of events in trace caseID 0 :3


4

In [0]:
# outputs the number of traces where the number of events in the particular trace is the index of below Series(CaseID).
# pd.value_counts(data['CaseID']).value_counts()


In [510]:
actS = pd.Series(data['ActivityID'] )
actS.value_counts()
classes = actS.max()
#zero will be for end of trace
print("number of classes(zero not included which will be denoted later as end of trace) :" +str(classes))
actS.value_counts()

number of classes(zero not included which will be denoted later as end of trace) :6


3    9089
5    6641
6    5890
1    5127
4    2482
2     181
Name: ActivityID, dtype: int64

In [0]:
#converting it into datetime datatype of pandas for easy access of date, time and other operations.
data['datetime'] = pd.to_datetime(data['CompleteTimestamp'])
data.drop(['CompleteTimestamp'], axis=1, inplace=True)
# data.head()


In [512]:
data.head()

,CaseID,ActivityID,datetime
0,0,3,2011-10-01 19:45:13
1,0,5,2011-10-01 20:17:08
2,0,6,2011-10-13 18:37:37
3,3,3,2011-10-01 19:43:13
4,3,5,2011-10-01 22:36:25


In [513]:
#set finds the unique numbers in caseID i.e list all the unique caseIDs.
ca_uni = list(sorted(set(data['CaseID'].values)))
ca_uni
a[0]

3

In [0]:
# data.head()

In [0]:
#datetime is shifted to find elapsed or concerned type of time further down the line.
data['dapa'] = data['datetime'].shift(1)
# data.head()

In [0]:
#difference of datetime is timedelta type in pandas which has a function called .total_seconds()
data['elapsed'] = data['datetime'] - data['dapa']
b = [x.total_seconds() for x in data['elapsed'] ]
# data['elapsed'].head()
data['elap'] = b
data.drop(columns=['elapsed','dapa'],inplace = True)
# data.head(50)

# data.head(5)

In [0]:
#case at all times denote the index of data panda, and ca_uni contains unique caseIDs, we increment case with a[i] amount i.e the number of events in that particular trace denoted by i (element of ca_uni).
case = 0
for i in ca_uni:
  data.loc[case,'elap'] = 0
  case += a[ i ]
  
# data.head(10)
  

In [0]:
#converted the seconds into days
data['elap'] = data['elap']/86400
# data.head()

In [0]:
#the trace is made into a string and named sequence abstraction
case = 0
for i in ca_uni:
#   data.loc[case,'elap'] = 0
  prev = data.loc[case,'ActivityID']
  prev = str(prev)
  data.loc[case,'seq'] =  prev
#   print(prev)
  case +=1
  for j in range(a[i]-1):
    new = data.loc[case,'ActivityID']
    prev = prev + str(new)
    data.loc[case,'seq'] = prev
    case +=1
  
# data.head(10)
  

In [0]:
#bag abstraction is made
li = ["".join(sorted(set(st), key=st.index)) for st in data['seq'] ]
data['bag'] = li
count = ["".join( [chr(row['seq'].count(ch)+96) for ch in row['bag']] )+row['bag'] for index,row in data.iterrows()]
data['bag'] = count
# data.head(50)

In [0]:
#set abstraction is made
tra1 = ["".join(sorted(set(st), key=st.index)) for st in data['seq'] ]
# tra2 = ["".join(sorted(st)) for st in tra1]
data['set'] = tra1
# data.head(20)

In [0]:
#future activity and the future elapsed time is brought on the same row
data['future'] = data['ActivityID'].shift(-1)
data['future_elap'] = data['elap'].shift(-1)
# data.head(20)

In [0]:
#at the change of trace, values are adjusted
case = -1
for i in ca_uni:
  data.loc[case,'future'] = 0
  data.loc[case,'future_elap'] = 0
#   print(i)
  case += a[ i ]


In [0]:
data.dropna(inplace = True)

**PREFIX DECIDE**

In [0]:
prefix = 2

In [0]:
#actual_time contains future elapsed time, only the reasonable data rows are kept, rest are dropped
actual_time = []
for index,row in data.iterrows():
  if( len(row['seq']) != prefix ):
      data = data.drop(index)
  else:
      actual_time.append(row['future_elap'])
  

In [0]:
data.reset_index(inplace = True)

In [536]:
till = int(data.shape[0]*0.80)
till

767

In [0]:
#data is separated for test and train 
testdata = data.iloc[till:]
data = data.iloc[:till]
actual_time_train = actual_time[:till]
actual_time_test = actual_time[till:]
# list(actual_time)

In [0]:
from statistics import mean
from sklearn.metrics import mean_absolute_error

                                            **SET ABSTRACTION**

In [0]:
setdic = {}

In [0]:
#a dictionary is made clustering similar sequences
for index, row in data.iterrows():
  if row['set'] in setdic.keys():
    setdic[row['set']][0].append(row['future'])
    setdic[row['set']][1].append(row['elap'])
  else:
    setdic[row['set']] = [[],[]]
    setdic[row['set']][0].append(row['future'])
    setdic[row['set']][1].append(row['elap'])

    
    

In [0]:
setpred = {}

In [0]:
# actunk and timeunk is found for unseen sequences if occurred, 
# setpred first element is most occurred element and second element is average elapsed time
actunk = []
timeunk = []
for key, value in setdic.items():
  setpred[key] = [max(set(value[0]), key = value[0].count), mean(value[1])]
  actunk.append(setpred[key][0])
  timeunk.append(setpred[key][1])

In [548]:
# actunk

[0.0, 0.0, 4.0, 6.0, 6.0, 4.0, 0.0, 0.0, 0.0, 0.0]

In [549]:
#most often occurred activity is unkact
unkact = max(actunk,key=actunk.count)


0.0

In [550]:
# unktime is the mean of all time given
unktime = mean(timeunk)


4.66954886809479

In [0]:
#predact and predtime is prepared to predict future similar sequence if occured
predact = []
predtime = []
for index, row in data.iterrows():
  predact.append(setpred[row['set']][0])
  predtime.append(setpred[row['set']][1])
  
  
actualact = list(data['future'])

  

In [552]:
trainaccuracy = sum(1 for x,y in zip(predact,actualact) if x == y) / float(len(actualact))
trainaccuracy

0.5058670143415906

In [553]:
trainmae =  mean_absolute_error(predtime,actual_time_train)
trainmae

2.920374782882207

In [0]:
#we predict by looking for similar occurrence of the test trace sequence, if new sequence is seen predict unktime unkact
predact = []
predtime = []
for index, row in testdata.iterrows():
  subset = row['set']
  if (row['set'] in setpred.keys()):
    predact.append(setpred[row['set']][0])
    predtime.append(setpred[row['set']][1])
    
  else:
    predact.append(unkact)  
    predtime.append(unktime) 
  
  
actualact = list(testdata['future'])


In [555]:
testaccuracy = sum(1 for x,y in zip(predact,actualact) if x == y) / float(len(actualact))
testaccuracy

0.4322916666666667

In [556]:
testmae =  mean_absolute_error(predtime,actual_time_test)
testmae

2.727193903033617

                                        **SEQUENCE ABSTRACTION**

In [0]:
seqdic= {}

In [0]:
for index, row in data.iterrows():
  if row['seq'] in seqdic.keys():
    seqdic[row['seq']][0].append(row['future'])
    seqdic[row['seq']][1].append(row['elap'])
  else:
    seqdic[row['seq']] = [[],[]]
    seqdic[row['seq']][0].append(row['future'])
    seqdic[row['seq']][1].append(row['elap'])

    
    

In [0]:
seqpred = {}

In [0]:
actunk = []
timeunk = []
for key, value in seqdic.items():
  seqpred[key] = [max(set(value[0]), key = value[0].count), mean(value[1])]
  actunk.append(seqpred[key][0])
  timeunk.append(seqpred[key][1])

In [0]:
unkact = max(actunk,key=actunk.count)

In [562]:
unktime = mean(timeunk)
unktime

4.234861576977737

In [0]:
predact = []
predtime = []
for index, row in data.iterrows():
  predact.append(seqpred[row['seq']][0])
  predtime.append(seqpred[row['seq']][1])
  
  
actualact = list(data['future'])


In [564]:
trainaccuracy = sum(1 for x,y in zip(predact,actualact) if x == y) / float(len(actualact))
trainaccuracy

0.6962190352020861

In [565]:
trainmae =  mean_absolute_error(predtime,actual_time_train)
trainmae

3.006222994968061

In [566]:
len(predact)

767

In [0]:
predact = []
predtime = []
for index, row in testdata.iterrows():
  subset = row['seq']
  if (row['seq'] in seqpred.keys()):
    predact.append(seqpred[row['seq']][0])
    predtime.append(seqpred[row['seq']][1])
  else:
    while(subset):
      subset = subset[1:]
      if (not subset):
        predact.append(unkact)
        break
      elif (subset in seqpred.keys()):
        predact.append(seqpred[subset][0])
        break
    predtime.append(unktime)
      
  
actualact = list(testdata['future'])


In [568]:
testaccuracy = sum(1 for x,y in zip(predact,actualact) if x == y) / float(len(actualact))
testaccuracy

0.6770833333333334

In [569]:
testmae =  mean_absolute_error(predtime,actual_time_test)
testmae

2.8473094685163622

                                        **BAG ABSTRACTION**

check if bag is right or not like sequence matters or not a6b5c4 and b5a6c4

In [0]:
bagdic= {}

In [0]:
for index, row in data.iterrows():
  if row['bag'] in bagdic.keys():
    bagdic[row['bag']][0].append(row['future'])
    bagdic[row['bag']][1].append(row['elap'])
  else:
    bagdic[row['bag']] = [[],[]]
    bagdic[row['bag']][0].append(row['future'])
    bagdic[row['bag']][1].append(row['elap'])

    
    

In [0]:
bagpred = {}

In [0]:
actunk = []
timeunk = []
for key, value in bagdic.items():
  bagpred[key] = [max(set(value[0]), key = value[0].count), mean(value[1])]
  actunk.append(bagpred[key][0])
  timeunk.append(bagpred[key][1])

In [0]:
unkact = max(actunk,key=actunk.count)

In [575]:
unktime = mean(timeunk)
unktime

4.234861576977737

In [0]:
predact = []
predtime = []
for index, row in data.iterrows():
  predact.append(bagpred[row['bag']][0])
  predtime.append(bagpred[row['bag']][1])
  
  
actualact = list(data['future'])


In [577]:
trainaccuracy = sum(1 for x,y in zip(predact,actualact) if x == y) / float(len(actualact))
trainaccuracy

0.6962190352020861

In [578]:
trainmae =  mean_absolute_error(predtime,actual_time_train)
trainmae

3.006222994968061

In [0]:
predact = []
predtime = []
for index, row in testdata.iterrows():
  subset = row['bag']
  if (row['bag'] in bagpred.keys()):
    predact.append(bagpred[row['bag']][0])
    predtime.append(bagpred[row['bag']][1])
  else:
    while(subset):
      subset = subset[1:]
      if (not subset):
        predact.append(unkact)
        break
      elif (subset in bagpred.keys()):
        predact.append(bagpred[subset][0])
        break
    predtime.append(unktime)
  
actualact = list(testdata['future'])


In [580]:
testaccuracy = sum(1 for x,y in zip(predact,actualact) if x == y) / float(len(actualact))
testaccuracy

0.6770833333333334

In [581]:
testmae =  mean_absolute_error(predtime,actual_time_test)
testmae

2.8473094685163622